In [44]:
import nltk, collections
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

frequencies = collections.Counter()
with open('samplefile.txt') as book:
    print book
    read_book = book.read()
words = nltk.word_tokenize(read_book)
print words

<open file 'samplefile.txt', mode 'r' at 0x000000000A968DB0>
['Data', 'science', ',', 'also', 'known', 'as', 'data-driven', 'science', ',', 'is', 'an', 'interdisciplinary', 'field', 'about', 'scientific', 'methods', ',', 'processes', ',', 'and', 'systems', 'to', 'extract', 'knowledge', 'or', 'insights', 'from', 'data', 'in', 'various', 'forms', ',', 'either', 'structured', 'or', 'unstructured', ',', 'similar', 'to', 'data', 'mining', '.', 'Data', 'science', 'is', 'a', 'concept', 'to', 'unify', 'statistics', ',', 'data', 'analysis', 'and', 'their', 'related', 'methods', 'in', 'order', 'to', 'understand', 'and', 'analyze', 'actual', 'phenomena', 'with', 'data', '.', 'It', 'employs', 'techniques', 'and', 'theories', 'drawn', 'from', 'many', 'fields', 'within', 'the', 'broad', 'areas', 'of', 'mathematics', ',', 'statistics', ',', 'information', 'science', ',', 'and', 'computer', 'science', ',', 'in', 'particular', 'from', 'the', 'subdomains', 'of', 'machine', 'learning', ',', 'classificati

In [51]:
tagged = nltk.pos_tag(words)

wrds = filter(lambda (word, tag): tag != 'VB', tagged)
#Count of all words
print len(tagged)
#count of words after removing Verbs 
print len(wrds)


447
430


In [54]:
wordlist = []
for w in wrds:
    wordlist.append(w[0])
print wordlist

['Data', 'science', ',', 'also', 'known', 'as', 'data-driven', 'science', ',', 'is', 'an', 'interdisciplinary', 'field', 'about', 'scientific', 'methods', ',', 'processes', ',', 'and', 'systems', 'to', 'knowledge', 'or', 'insights', 'from', 'data', 'in', 'various', 'forms', ',', 'either', 'structured', 'or', 'unstructured', ',', 'similar', 'to', 'mining', '.', 'Data', 'science', 'is', 'a', 'concept', 'to', 'statistics', ',', 'data', 'analysis', 'and', 'their', 'related', 'methods', 'in', 'order', 'to', 'and', 'actual', 'phenomena', 'with', 'data', '.', 'It', 'employs', 'techniques', 'and', 'theories', 'drawn', 'from', 'many', 'fields', 'within', 'the', 'broad', 'areas', 'of', 'mathematics', ',', 'statistics', ',', 'information', 'science', ',', 'and', 'computer', 'science', ',', 'in', 'particular', 'from', 'the', 'subdomains', 'of', 'machine', 'learning', ',', 'classification', ',', 'cluster', 'analysis', ',', 'data', 'mining', ',', 'databases', ',', 'and', 'visualization', '.', 'Turin

In [55]:
stop_words = set(stopwords.words("english"))
filtered_words = []

for w in wordlist:
    w.lower()
    if w not in stop_words:
        filtered_words.append(w)
print filtered_words


['Data', 'science', ',', 'also', 'known', 'data-driven', 'science', ',', 'interdisciplinary', 'field', 'scientific', 'methods', ',', 'processes', ',', 'systems', 'knowledge', 'insights', 'data', 'various', 'forms', ',', 'either', 'structured', 'unstructured', ',', 'similar', 'mining', '.', 'Data', 'science', 'concept', 'statistics', ',', 'data', 'analysis', 'related', 'methods', 'order', 'actual', 'phenomena', 'data', '.', 'It', 'employs', 'techniques', 'theories', 'drawn', 'many', 'fields', 'within', 'broad', 'areas', 'mathematics', ',', 'statistics', ',', 'information', 'science', ',', 'computer', 'science', ',', 'particular', 'subdomains', 'machine', 'learning', ',', 'classification', ',', 'cluster', 'analysis', ',', 'data', 'mining', ',', 'databases', ',', 'visualization', '.', 'Turing', 'award', 'winner', 'Jim', 'Gray', 'imagined', 'data', 'science', 'fourth', 'paradigm', 'science', '(', 'empirical', ',', 'theoretical', ',', 'computational', 'data-driven', ')', 'asserted', '``', '

In [56]:
for w in filtered_words:
    w.replace(" ", "")
print filtered_words

['Data', 'science', ',', 'also', 'known', 'data-driven', 'science', ',', 'interdisciplinary', 'field', 'scientific', 'methods', ',', 'processes', ',', 'systems', 'knowledge', 'insights', 'data', 'various', 'forms', ',', 'either', 'structured', 'unstructured', ',', 'similar', 'mining', '.', 'Data', 'science', 'concept', 'statistics', ',', 'data', 'analysis', 'related', 'methods', 'order', 'actual', 'phenomena', 'data', '.', 'It', 'employs', 'techniques', 'theories', 'drawn', 'many', 'fields', 'within', 'broad', 'areas', 'mathematics', ',', 'statistics', ',', 'information', 'science', ',', 'computer', 'science', ',', 'particular', 'subdomains', 'machine', 'learning', ',', 'classification', ',', 'cluster', 'analysis', ',', 'data', 'mining', ',', 'databases', ',', 'visualization', '.', 'Turing', 'award', 'winner', 'Jim', 'Gray', 'imagined', 'data', 'science', 'fourth', 'paradigm', 'science', '(', 'empirical', ',', 'theoretical', ',', 'computational', 'data-driven', ')', 'asserted', '``', '

In [57]:
#Removal of puncutation marks
import string
filtered_words = [''.join(c for c in s if c not in string.punctuation) for s in filtered_words]
print filtered_words

['Data', 'science', '', 'also', 'known', 'datadriven', 'science', '', 'interdisciplinary', 'field', 'scientific', 'methods', '', 'processes', '', 'systems', 'knowledge', 'insights', 'data', 'various', 'forms', '', 'either', 'structured', 'unstructured', '', 'similar', 'mining', '', 'Data', 'science', 'concept', 'statistics', '', 'data', 'analysis', 'related', 'methods', 'order', 'actual', 'phenomena', 'data', '', 'It', 'employs', 'techniques', 'theories', 'drawn', 'many', 'fields', 'within', 'broad', 'areas', 'mathematics', '', 'statistics', '', 'information', 'science', '', 'computer', 'science', '', 'particular', 'subdomains', 'machine', 'learning', '', 'classification', '', 'cluster', 'analysis', '', 'data', 'mining', '', 'databases', '', 'visualization', '', 'Turing', 'award', 'winner', 'Jim', 'Gray', 'imagined', 'data', 'science', 'fourth', 'paradigm', 'science', '', 'empirical', '', 'theoretical', '', 'computational', 'datadriven', '', 'asserted', '', 'everything', 'science', 'ch

In [58]:
import string
processed_word = []
for each in filtered_words:
    if not each:
        continue
    else:
        processed_word.append(each) 
print processed_word

['Data', 'science', 'also', 'known', 'datadriven', 'science', 'interdisciplinary', 'field', 'scientific', 'methods', 'processes', 'systems', 'knowledge', 'insights', 'data', 'various', 'forms', 'either', 'structured', 'unstructured', 'similar', 'mining', 'Data', 'science', 'concept', 'statistics', 'data', 'analysis', 'related', 'methods', 'order', 'actual', 'phenomena', 'data', 'It', 'employs', 'techniques', 'theories', 'drawn', 'many', 'fields', 'within', 'broad', 'areas', 'mathematics', 'statistics', 'information', 'science', 'computer', 'science', 'particular', 'subdomains', 'machine', 'learning', 'classification', 'cluster', 'analysis', 'data', 'mining', 'databases', 'visualization', 'Turing', 'award', 'winner', 'Jim', 'Gray', 'imagined', 'data', 'science', 'fourth', 'paradigm', 'science', 'empirical', 'theoretical', 'computational', 'datadriven', 'asserted', 'everything', 'science', 'changing', 'impact', 'information', 'technology', 'data', 'deluge', 'When', 'Harvard', 'Business',

In [59]:
#Lemmatization addin u in front of certain words
lemma_word = []
for w in processed_word:
    lemma_word.append(wnl.lemmatize(w))
print lemma_word

['Data', 'science', 'also', 'known', 'datadriven', 'science', 'interdisciplinary', 'field', 'scientific', u'method', u'process', u'system', 'knowledge', u'insight', 'data', 'various', u'form', 'either', 'structured', 'unstructured', 'similar', 'mining', 'Data', 'science', 'concept', u'statistic', 'data', 'analysis', 'related', u'method', 'order', 'actual', u'phenomenon', 'data', 'It', u'employ', u'technique', u'theory', 'drawn', 'many', u'field', 'within', 'broad', u'area', 'mathematics', u'statistic', 'information', 'science', 'computer', 'science', 'particular', 'subdomains', 'machine', 'learning', 'classification', 'cluster', 'analysis', 'data', 'mining', u'database', 'visualization', 'Turing', 'award', 'winner', 'Jim', 'Gray', 'imagined', 'data', 'science', 'fourth', 'paradigm', 'science', 'empirical', 'theoretical', 'computational', 'datadriven', 'asserted', 'everything', 'science', 'changing', 'impact', 'information', 'technology', 'data', 'deluge', 'When', 'Harvard', 'Business',

In [60]:
#stemming add more u in front of words
from nltk.stem import PorterStemmer
ps = PorterStemmer()
#Stemming
stemmed_words = []
for w in processed_word:
    stemmed_words.append(ps.stem(w))
print stemmed_words

['data', u'scienc', 'also', 'known', 'datadriven', u'scienc', u'interdisciplinari', 'field', u'scientif', u'method', u'process', u'system', u'knowledg', u'insight', 'data', u'variou', u'form', 'either', u'structur', u'unstructur', 'similar', u'mine', 'data', u'scienc', 'concept', u'statist', 'data', u'analysi', u'relat', u'method', 'order', 'actual', 'phenomena', 'data', 'It', u'employ', u'techniqu', u'theori', 'drawn', u'mani', u'field', 'within', 'broad', u'area', u'mathemat', u'statist', u'inform', u'scienc', u'comput', u'scienc', 'particular', u'subdomain', u'machin', u'learn', u'classif', 'cluster', u'analysi', 'data', u'mine', u'databas', u'visual', u'ture', 'award', 'winner', 'jim', 'gray', u'imagin', 'data', u'scienc', 'fourth', 'paradigm', u'scienc', u'empir', u'theoret', u'comput', 'datadriven', u'assert', u'everyth', u'scienc', u'chang', 'impact', u'inform', u'technolog', 'data', u'delug', 'when', 'harvard', u'busi', 'review', u'call', 'the', 'sexiest', 'job', '21st', u'cent

In [61]:

for w in processed_word:
    frequencies[w] += 1
 
bigram = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(processed_word)
#To find frequently together occuring 2 words
finder.apply_freq_filter(2)
  
print ('Those are the words and their frequency of occurrence in the book:')
print (frequencies) 

print ('Those are the 10 most bigrams occurring in the book:')
print (finder.nbest(bigram.pmi, 10))
#print (read_book.index('data'))
#print (read_book.index('Assisted Reporting'))

Those are the words and their frequency of occurrence in the book:
Counter({'data': 20, 'science': 12, 'Data': 6, 'term': 3, 'scientists': 3, 'many': 3, 'statistics': 3, 'The': 3, 'programs': 3, 'analysis': 3, 'sources': 2, '21st': 2, 'Business': 2, 'like': 2, 'skills': 2, 'methods': 2, 'current': 2, 'Job': 2, 'Harvard': 2, 'Review': 2, 'Sexiest': 2, 'often': 2, 'use': 2, 'mining': 2, 'offering': 2, 'present': 2, 'Century': 2, 'datadriven': 2, 'technology': 2, 'information': 2, 'also': 2, 'rather': 2, 'concept': 1, 'surrounding': 1, 'Jim': 1, 'results': 1, 'bandwidth': 1, 'scientist': 1, 'including': 1, 'imagined': 1, 'occupation': 1, 'impact': 1, 'visualization': 1, 'rapid': 1, 'knowledge': 1, 'field': 1, 'studentpaid': 1, 'Company': 1, 'winner': 1, 'forms': 1, 'systems': 1, 'rich': 1, 'insight': 1, 'They': 1, 'interdisciplinary': 1, 'When': 1, 'processes': 1, 'unstructured': 1, 'While': 1, 'papersreports': 1, 'despite': 1, 'using': 1, 'new': 1, 'areas': 1, 'Because': 1, 'fields': 1, 